In [1]:
import h5py
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
from scipy.signal import argrelextrema
from scipy import stats
import glob, os
import datetime as dt
from scipy.signal import find_peaks, peak_prominences
import seaborn as sns

In [2]:
def unnesting(df, explode):
    idx = df.index.repeat(df[explode[0]].str.len())
    df1 = pd.concat([
        pd.DataFrame({x: np.concatenate(df[x].values)}) for x in explode], axis=1)
    df1.index = idx

    return df1.join(df.drop(explode, 1), how='left')

In [90]:
subjects = ["MC71"]
for subject in subjects:
    base = "/home/ml/Documents/TADSS_Videos/"
    folder = base+subject+"/cut/dlc_output_16-810"
    folder2 = base+subject+"/processed/"
    #empty list of files
    files = []
    os.chdir(folder)
    if subject == "MC71":
        injured = "Front Left"
    elif subject == "BB03":
        injured = "Front Right"
    else:
        print("Subject not found")
    for file in glob.glob("*.h5"):
        files.append(file)
        #print(file)
    peaks = []
    peakcols = ["subject","date","run","peak_fr_x","peak_fr_y","likelihood_fr","peak_fl_x","peak_fl_y","likelihood_fl","peak_br_x","peak_br_y","likelihood_br","peak_bl_x","peak_bl_y","likelihood_bl","z-score_fr","z-score_fl","z-score_br","z-score_bl"]
    xdist = -25
    likes_fr = []
    likes_fl = []
    likes_bl = []
    likes_br = []
    
    for data in files:
        subject = data.split("_")[0]
        date = data.split("_")[1]
        if len(data.split("_")) == 10:
            run = data.split("_")[2]
        else:
            run = data.split("_")[3]
        df = pd.read_hdf(data)
        pred = 0.1

        #frontright
        df_frw = df['DLC_resnet101_LadderWalkFeb13shuffle1_1030000']['right wrist']
        df_frf = df['DLC_resnet101_LadderWalkFeb13shuffle1_1030000']['right fingers']
        #frontleft
        df_flw = df['DLC_resnet101_LadderWalkFeb13shuffle1_1030000']['left wrist']
        df_flf = df['DLC_resnet101_LadderWalkFeb13shuffle1_1030000']['left fingers']
        #backright
        df_bra = df['DLC_resnet101_LadderWalkFeb13shuffle1_1030000']['right ankle']
        df_brt = df['DLC_resnet101_LadderWalkFeb13shuffle1_1030000']['right toes']
        #frontleft
        df_bla = df['DLC_resnet101_LadderWalkFeb13shuffle1_1030000']['left ankle']
        df_blt = df['DLC_resnet101_LadderWalkFeb13shuffle1_1030000']['left toes']
        
        
        #filter each point with likelihoods
        df_frw = df_frw.loc[df_frw['likelihood']>pred]
        df_frw = df_frw.reset_index()
        df_frw.name = "Right Wrist"
        
        df_frf = df_frf.loc[df_frf['likelihood']>pred]
        df_frf = df_frw.reset_index()
        df_frf.name = "Right Fingers"
        
        df_flw = df_flw.loc[df_flw['likelihood']>pred]
        df_flw = df_flw.reset_index()
        df_flw.name = "Left Wrist"
        
        df_flf = df_flf.loc[df_flf['likelihood']>pred]
        df_flf = df_flw.reset_index()
        df_flf.name = "Left Fingers"
        
        df_bra = df_bra.loc[df_bra['likelihood']>pred]
        df_bra = df_bra.reset_index()
        df_bra.name = "Right Ankle"
        
        df_brt = df_brt.loc[df_brt['likelihood']>pred]
        df_brt = df_bra.reset_index()
        df_brt.name = "Right Toes"
        
        df_bla = df_bla.loc[df_bla['likelihood']>pred]
        df_bla = df_bla.reset_index()
        df_bla.name = "Left Ankle"
        
        df_blt = df_blt.loc[df_blt['likelihood']>pred]
        df_blt = df_bla.reset_index()
        df_blt.name = "Left Toes"
        
        #find x and y coordinates
        #right
        x_frw = df_frw["x"]
        y_frw = df_frw["y"]
        
        x_frf = df_frf["x"]
        y_frf = df_frf["y"]
        
        x_bra = df_bra["x"]
        y_bra = df_bra["y"]
        
        x_brt = df_brt["x"]
        y_brt = df_brt["y"]
        
        #left
        x_flw = df_flw["x"]
        y_flw = df_flw["y"]
        
        x_flf = df_flf["x"]
        y_flf = df_flf["y"]
        
        x_bla = df_bla["x"]
        y_bla = df_bla["y"]
        
        x_blt = df_blt["x"]
        y_blt = df_blt["y"]
        
        promperc = 55
        #highest is 85
        
        #find prominences of local minima
        peaks_frw = find_peaks(y_frw,prominence=0)
        proms_frw = peaks_frw[1]['prominences']
        if len(proms_frw)>=1:
            prom_frw = np.percentile(proms_frw,promperc).min()
        else:
            prom_frw = 0
            
        peaks_frf = find_peaks(y_frf,prominence=0)
        proms_frf = peaks_frf[1]['prominences']
        if len(proms_frf)>=1:
            prom_frf = np.percentile(proms_frf,promperc).min()
        else:
            prom_frf = 0
            
        peaks_flw = find_peaks(y_flw,prominence=0)
        proms_flw = peaks_flw[1]['prominences']
        if len(proms_flw)>=1:
            prom_flw = np.percentile(proms_flw,promperc).min()
        else:
            prom_flw = 0
            
        peaks_flf = find_peaks(y_flf,prominence=0)
        proms_flf = peaks_flf[1]['prominences']
        if len(proms_flf)>=1:
            prom_flf = np.percentile(proms_flf,promperc).min()
        else:
            prom_flf = 0
            
        peaks_bra = find_peaks(y_bra,prominence=0)
        proms_bra = peaks_bra[1]['prominences']
        if len(proms_bra)>=1:
            prom_bra = np.percentile(proms_bra,promperc).min()
        else:
            prom_bra = 0
            
        peaks_brt = find_peaks(y_brt,prominence=0)
        proms_brt = peaks_brt[1]['prominences']
        if len(proms_brt)>=1:
            prom_brt = np.percentile(proms_brt,promperc).min()
        else:
            prom_brt = 0
            
        peaks_bla = find_peaks(y_bla,prominence=0)
        proms_bla = peaks_bla[1]['prominences']
        if len(proms_bla)>=1:
            prom_bla = np.percentile(proms_bla,promperc).min()
        else:
            prom_bla = 0
            
        peaks_blt = find_peaks(y_blt,prominence=0)
        proms_blt = peaks_blt[1]['prominences']
        if len(proms_blt)>=1:
            prom_blt = np.percentile(proms_blt,promperc).min()
        else:
            prom_blt = 0        
        
        #y distance below the median for minimum height of peak. in pixels
        dh = 25

        #take the peaks
        med_frw = np.median(y_frw)
        peaks_frw= find_peaks(y_frw, height = med_frw-dh, prominence=prom_frw)
        med_frf = np.median(y_frf)
        peaks_frf= find_peaks(y_frf, height = med_frf-dh, prominence=prom_frf)

        med_flw = np.median(y_flw)
        peaks_flw= find_peaks(y_flw, height = med_flw-dh, prominence=prom_flw)
        med_flf = np.median(y_flf)
        peaks_flf= find_peaks(y_flf, height = med_flf-dh, prominence=prom_flf)

        med_bra = np.median(y_bra)
        peaks_bra= find_peaks(y_bra, height = med_bra-dh, prominence=prom_bra)
        med_brt = np.median(y_brt)
        peaks_brt= find_peaks(y_brt, height = med_brt-dh, prominence=prom_brt)

        med_bla = np.median(y_bla)
        peaks_bla= find_peaks(y_bla, height = med_bla-dh, prominence=prom_bla)
        med_blt = np.median(y_blt)
        peaks_blt= find_peaks(y_blt, height = med_blt-dh, prominence=prom_blt)
        
        #calculate zscore for each peak
        zsc_frw = (np.abs(stats.zscore(y_frw[peaks_frw[0]])))
        zsc_frf = (np.abs(stats.zscore(y_frf[peaks_frf[0]])))
        
        zsc_flw = (np.abs(stats.zscore(y_flw[peaks_flw[0]])))
        zsc_flf = (np.abs(stats.zscore(y_flf[peaks_flf[0]])))
        
        zsc_bra = (np.abs(stats.zscore(y_bra[peaks_bra[0]])))
        zsc_brt = (np.abs(stats.zscore(y_brt[peaks_brt[0]])))
        
        zsc_bla = (np.abs(stats.zscore(y_bla[peaks_bla[0]])))
        zsc_blt = (np.abs(stats.zscore(y_blt[peaks_blt[0]])))
        
        peaks.append([subject,date,run,x_frw[peaks_frw[0]],y_frw[peaks_frw[0]],x_frf[peaks_frf[0]],y_frf[peaks_frf[0]],x_flw[peaks_flw[0]],y_flw[peaks_flw[0]],x_flf[peaks_flf[0]],y_flf[peaks_flf[0]],x_bra[peaks_bra[0]],y_bra[peaks_bra[0]],x_brt[peaks_brt[0]],y_brt[peaks_brt[0]],x_bla[peaks_bla[0]],y_bla[peaks_bla[0]],x_blt[peaks_blt[0]],y_blt[peaks_blt[0]],zsc_frw,zsc_frf,zsc_flw,zsc_flf,zsc_bra,zsc_brt,zsc_bla,zsc_blt])



    '''fr = unnesting(df2[["subject","daterun","peak_fr_x","peak_fr_y","likelihood_fr","z-score_fr"]],['peak_fr_x','peak_fr_y',"likelihood_fr","z-score_fr"])
    fr = fr[["subject","daterun","peak_fr_x","peak_fr_y","likelihood_fr","z-score_fr"]]
    fr = fr.rename(columns={"daterun":"daterun","peak_fr_x":"x","peak_fr_y":"y","likelihood_fr":"likelihood","z-score_fr":"z-score"})
    fr["limb"] = "Front Right"
    fr = fr.reset_index()
    fr = fr.drop("index",axis=1)
    
    peak_df_fr = fr
    peak_df_fr = peak_df_fr.sort_values(["daterun","x"],ascending=False)
    peak_df_fr["diff"]=peak_df_fr.groupby(["subject","daterun","limb"])["x"].diff()
    peak_df_fr["keep"] = "Y"
    peak_df_fr.loc[(peak_df_fr["diff"])>xdist,"keep"] = "N"
    peak_df_fr = peak_df_fr.loc[peak_df_fr["keep"]=="Y"]
    

    fl = unnesting(df2[["subject",'daterun','peak_fl_x','peak_fl_y',"likelihood_fl","z-score_fl"]],['peak_fl_x','peak_fl_y',"likelihood_fl","z-score_fl"])
    fl = fl[["subject",'daterun','peak_fl_x','peak_fl_y',"likelihood_fl","z-score_fl"]]
    fl = fl.rename(columns={"daterun":"daterun","peak_fl_x":"x","peak_fl_y":"y","likelihood_fl":"likelihood","z-score_fl":"z-score"})
    fl["limb"] = "Front Left"
    fl = fl.reset_index()
    fl = fl.drop("index",axis=1)
    
    peak_df_fl = fl
    peak_df_fl = peak_df_fl.sort_values(["daterun","x"],ascending=False)
    peak_df_fl["diff"]=peak_df_fl.groupby(["subject","daterun","limb"])["x"].diff()
    peak_df_fl["keep"] = "Y"
    peak_df_fl.loc[(peak_df_fl["diff"])>xdist,"keep"] = "N"
    peak_df_fl = peak_df_fl.loc[peak_df_fl["keep"]=="Y"]

    br = unnesting(df2[["subject",'daterun','peak_br_x','peak_br_y',"likelihood_br","z-score_br"]],['peak_br_x','peak_br_y',"likelihood_br","z-score_br"])
    br = br[["subject",'daterun','peak_br_x','peak_br_y',"likelihood_br","z-score_br"]]
    br = br.rename(columns={"daterun":"daterun","peak_br_x":"x","peak_br_y":"y","likelihood_br":"likelihood","z-score_br":"z-score"})
    br["limb"] = "Back Right"
    br = br.reset_index()
    br = br.drop("index",axis=1)
    
    peak_df_br = br
    peak_df_br = peak_df_br.sort_values(["daterun","x"],ascending=False)
    peak_df_br["diff"]=peak_df_br.groupby(["subject","daterun","limb"])["x"].diff()
    peak_df_br["keep"] = "Y"
    peak_df_br.loc[(peak_df_br["diff"])>xdist,"keep"] = "N"
    peak_df_br = peak_df_br.loc[peak_df_br["keep"]=="Y"]

    bl = unnesting(df2[["subject",'daterun','peak_bl_x','peak_bl_y',"likelihood_bl","z-score_bl"]],['peak_bl_x','peak_bl_y',"likelihood_bl","z-score_bl"])
    bl = bl[["subject",'daterun','peak_bl_x','peak_bl_y',"likelihood_bl","z-score_bl"]]
    bl = bl.rename(columns={"daterun":"daterun","peak_bl_x":"x","peak_bl_y":"y","likelihood_bl":"likelihood","z-score_bl":"z-score"})
    bl["limb"] = "Back Left"
    bl = bl.reset_index()
    bl = bl.drop("index",axis=1)
    
    peak_df_bl = bl
    peak_df_bl = peak_df_bl.sort_values(["daterun","x"],ascending=False)
    peak_df_bl["diff"]=peak_df_bl.groupby(["subject","daterun","limb"])["x"].diff()
    peak_df_bl["keep"] = "Y"
    peak_df_bl.loc[(peak_df_bl["diff"]>xdist),"keep"] = "N"
    peak_df_bl = peak_df_bl.loc[peak_df_bl["keep"]=="Y"]
    
    concatlist = [peak_df_fr,peak_df_fl,peak_df_br,peak_df_bl]
    newdf = pd.concat(concatlist)
    #newdf.to_csv(folder2+"/"+subject+"_peaks.csv")
    
    zreq = 2
    zreq_injured = 2
    #was 0.6
    
    zdf = newdf.drop(["z-score"],axis = 1)
    #recalculate z-score of peaks using all the limbs
    zdf2 = zdf.groupby(['subject','daterun']).y.transform(lambda x : np.abs(stats.zscore(x)))
    zdf['z-score'] = zdf2
                                                                   
    hitdf = zdf.loc[ (zdf['z-score']<=zreq) | ((zdf['z-score']<=zreq_injured) & (zdf['limb']==injured))].groupby(['subject','daterun','limb'])['z-score'].count()
    #hitdf = hitdf.drop(['y','likelihood','limb','z-score'],axis=1)
    #hitdf = hitdf.rename(columns={'x':'dlc_hits'})
    slipdf = zdf.loc[ (zdf['z-score']>zreq) | ((zdf['z-score']>zreq_injured) & (zdf['limb']==injured))].groupby(['subject','daterun','limb'])['z-score'].count()
    #slipdf = slipdf.drop(['y','likelihood','limb','z-score'],axis=1)
    #slipdf = slipdf.rename(columns={'x':'dlc_miss'})
    #stepdf = zdf.groupby(['subject','daterun','limb'])['z-score'].count()
    #stepdf = stepdf.drop(['y','likelihood','limb','z-score'],axis=1)
    #stepdf = stepdf.rename(columns={'x':'dlc_steps'})
    
    
    
    
    scoredf = pd.DataFrame()
    scoredf['dlc_hits'] = hitdf
    scoredf['dlc_slips'] = slipdf
    scoredf = scoredf.fillna(0)
    scoredf['dlc_steps'] = scoredf['dlc_hits'].add(scoredf['dlc_slips'])
    scoredf = scoredf.reset_index()
    scoredf[['date',"run"]] = scoredf.daterun.str.split(expand=True)
    scoredf = scoredf.drop("daterun",axis=1)
    scoredf = scoredf[["subject","date","run","limb","dlc_hits","dlc_slips","dlc_steps"]]
    scoredf['date'] = pd.to_datetime(scoredf["date"])
    scoredf.to_csv(folder2 + "/"+subject+"_rescored_n2"+str(dt.datetime.now().time())+".csv")
    plt.close()'''

In [106]:
peakcols2 = ["subject","date","run","peak_fr_w_x","peak_fr_w_y","peak_fr_f_x","peak_fr_f_y","peak_fl_w_x","peak_fl_w_y","peak_fl_f_x","peak_fl_f_y","peak_br_a_x","peak_br_a_y","peak_br_t_x","peak_br_t_y","peak_bl_a_x","peak_bl_a_y","peak_bl_t_x","peak_bl_t_y","zscore_frw","zscore_frf","zscore_flw","zscore_flf","zscore_bra","zscore_brt","zscore_bla","zscore_blt"]
df2 = pd.DataFrame(peaks,columns=peakcols2)

In [86]:
df2

,subject,date,run,peak_fr_w_x,peak_fr_w_y,peak_fr_f_x,peak_fr_f_y,peak_fl_w_x,peak_fl_w_y,peak_fl_f_x,...,peak_bl_t_x,peak_bl_t_y,zscore_frw,zscore_frf,zscore_flw,zscore_flf,zscore_bra,zscore_brt,zscore_bla,zscore_blt
0,MC71,7-8-19,L3,8 1197.352051 12 1038.174927 14 973...,8 486.994415 12 487.504425 14 490.29...,8 1197.352051 12 1038.174927 14 973...,8 486.994415 12 487.504425 14 490.29...,10 1128.734009 21 718.216675 31 434...,10 489.937317 21 521.854370 31 509.01...,10 1128.734009 21 718.216675 31 434...,...,7 1356.861328 16 1116.055664 25 898...,7 486.058655 16 489.720184 25 499.50...,"[0.7361475460663834, 0.6996411072187736, 0.500...","[0.7361475460663834, 0.6996411072187736, 0.500...","[1.173531165036434, 0.22680395007539422, 0.336...","[1.173531165036434, 0.22680395007539422, 0.336...","[0.7323175557520024, 0.7775034941480529, 1.682...","[0.7323175557520024, 0.7775034941480529, 1.682...","[1.238894497109719, 0.6985675769372318, 0.7448...","[1.238894497109719, 0.6985675769372318, 0.7448..."
1,MC71,3-28-19,R4,"4 577.798828 16 1286.815063 Name: x, d...","4 693.135803 16 688.481079 Name: y, dty...","4 577.798828 16 1286.815063 Name: x, d...","4 693.135803 16 688.481079 Name: y, dty...","11 1057.872681 17 1441.297974 Name: x, d...","11 685.298645 17 676.016907 Name: y, dty...","11 1057.872681 17 1441.297974 Name: x, d...",...,"7 699.007874 12 1042.340942 Name: x, d...","7 680.447754 12 676.311707 Name: y, dty...","[1.0, 1.0]","[1.0, 1.0]","[1.0, 1.0]","[1.0, 1.0]","[1.0, 1.0]","[1.0, 1.0]","[1.0, 1.0]","[1.0, 1.0]"
2,MC71,3-26-19,L1,2 1529.804932 9 1240.742432 19 732...,2 343.265778 9 340.406494 19 337.72...,2 1529.804932 9 1240.742432 19 732...,2 343.265778 9 340.406494 19 337.72...,4 1414.588257 13 1091.828125 21 807...,4 346.092224 13 338.172546 21 337.73...,4 1414.588257 13 1091.828125 21 807...,...,18 1109.000122 28 831.110535 35 503...,18 333.221283 28 328.874268 35 343.97...,"[1.2374305855453094, 0.02577839944610139, 1.21...","[1.2374305855453094, 0.02577839944610139, 1.21...","[0.4636386555958948, 0.9034802691937092, 0.978...","[0.4636386555958948, 0.9034802691937092, 0.978...","[1.3501738102986984, 1.039467216757461, 0.3107...","[1.3501738102986984, 1.039467216757461, 0.3107...","[0.33651412167109523, 1.0213095343733698, 1.35...","[0.33651412167109523, 1.0213095343733698, 1.35..."
3,MC71,3-27-19,L2,6 1306.680908 9 1006.050903 13 820...,6 505.075989 9 504.521088 13 505.82...,6 1306.680908 9 1006.050903 13 820...,6 505.075989 9 504.521088 13 505.82...,9 1146.130859 16 823.421448 31 317...,9 508.275726 16 505.888123 31 525.17...,9 1146.130859 16 823.421448 31 317...,...,5 1497.796021 20 836.036377 27 539...,5 506.014313 20 503.197083 27 509.66...,"[0.594694085640507, 0.8866816795541619, 0.1981...","[0.594694085640507, 0.8866816795541619, 0.1981...","[0.5635098454564564, 0.8415627040164123, 1.405...","[0.5635098454564564, 0.8415627040164123, 1.405...","[1.2343946834012727, 0.5212576235136817, 0.301...","[1.2343946834012727, 0.5212576235136817, 0.301...","[0.10546029273827034, 1.1686046132245218, 1.27...","[0.10546029273827034, 1.1686046132245218, 1.27..."
4,MC71,7-11-19,R2,7 692.150452 14 660.732727 24 890...,7 559.895813 14 594.174255 24 555.71...,7 692.150452 14 660.732727 24 890...,7 559.895813 14 594.174255 24 555.71...,4 578.198730 18 881.420959 22 913...,4 560.873169 18 552.579346 22 545.39...,4 578.198730 18 881.420959 22 913...,...,3 448.895996 12 655.353333 22 730...,3 591.759705 12 597.179077 22 592.88...,"[0.27772429151681255, 2.2295199423031655, 0.03...","[0.27772429151681255, 2.2295199423031655, 0.03...","[1.6762415072527794, 0.7620233707279077, 0.029...","[1.6762415072527794, 0.7620233707279077, 0.029...","[0.9612784199528355, 0.6422349859367303, 0.824...","[0.9612784199528355, 0.6422349859367303, 0.824...","[0.8761998834998771, 1.085041247298129, 0.9194...","[0.8761998834998771, 1.085041247298129, 0.9194..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96,MC71,6-6

In [87]:
p_fr_w = unnesting(df2[["subject","date","run","peak_fr_w_x","peak_fr_w_y","zscore_frw"]],["peak_fr_w_x","peak_fr_w_y","zscore_frw"])
p_fr_f = unnesting(df2[["subject","date","run","peak_fr_f_x","peak_fr_f_y","zscore_frw"]],["peak_fr_f_x","peak_fr_f_y","zscore_frf"])

KeyError: 'zscore_frf'

In [70]:
p_fl_w = unnesting(df2[["subject","date","run","peak_fl_w_x","peak_fl_w_y","zscore_flw"]],["peak_fl_w_x","peak_fl_w_y","zscore_flw"])
p_fl_f = unnesting(df2[["subject","date","run","peak_fl_f_x","peak_fl_f_y","zscore_flf"]],["peak_fl_f_x","peak_fl_f_y","zscore_flf"])

In [52]:
p_br_a = unnesting(df2[["subject","date","run","peak_br_a_x","peak_br_a_y","zscore_bra"]],["peak_br_a_x","peak_br_a_y","zscore_bra"])
p_br_t = unnesting(df2[["subject","date","run","peak_br_t_x","peak_br_t_y","zscore_brt"]],["peak_br_t_x","peak_br_t_y","zscore_brt"])

In [53]:
p_bl_a = unnesting(df2[["subject","date","run","peak_bl_a_x","peak_bl_a_y","zscore_bla"]],["peak_bl_a_x","peak_bl_a_y","zscore_bla"])
p_bl_t = unnesting(df2[["subject","date","run","peak_bl_t_x","peak_bl_t_y","zscore_blt"]],["peak_bl_t_x","peak_bl_t_y","zscore_blt"])

In [111]:
all_peaks_df = unnesting(df2[peakcols2],peakcols2[3:27])

In [112]:
all_peaks_df.to_csv("/home/ml/Documents/TADSS_Videos/MC71/peaks.csv")


In [65]:
prom_frw

0.95574951171875

In [113]:
unnesting(df2.loc[df2["date"]=='6-4-19'].loc[df2['run']=="R1"][peakcols2],peakcols2[3:27]).to_csv("/home/ml/Documents/TADSS_Videos/MC71/peaks_6-4-19_R1.csv")

In [109]:
peakcols2[3:27]

['peak_fr_w_x',
 'peak_fr_w_y',
 'peak_fr_f_x',
 'peak_fr_f_y',
 'peak_fl_w_x',
 'peak_fl_w_y',
 'peak_fl_f_x',
 'peak_fl_f_y',
 'peak_br_a_x',
 'peak_br_a_y',
 'peak_br_t_x',
 'peak_br_t_y',
 'peak_bl_a_x',
 'peak_bl_a_y',
 'peak_bl_t_x',
 'peak_bl_t_y',
 'zscore_frw',
 'zscore_frf',
 'zscore_flw',
 'zscore_flf',
 'zscore_bra',
 'zscore_brt',
 'zscore_bla',
 'zscore_blt']